## Multi-Agent Orchestration: Acura MDX Winterization Workflow

This notebook implements a Retrieval-Augmented Generation (RAG) pipeline to deliver winterization recommendations for the 2022 Acura MDX. It reindexes the owner's manual using 768-dimensional embeddings and orchestrates multiple AI agents to generate actionable outputs using Google Cloud infrastructure.

### Summary of Fix

Earlier attempts failed due to dimensionality mismatches:
> Precondition failed: Query dimensionality (768) does not match database dimensionality (384)

This version resolves that by:
- Switching to the latest `text-embedding-005` model with 768-dimensional output
- Creating a new Vertex AI Matching Engine index and endpoint in `us-central1`
- Uploading correctly formatted JSON datapoints with `id`, `embedding`, and `content`

### Technologies Used

- **Vertex AI**: Hosts the vector index and serves queries via Matching Engine
- **LlamaIndex**: Handles document chunking, embedding, and interfacing with the vector store
- **Crew AI**: Provides multi-agent orchestration with role-based tasks
- **Gemini LLMs**: Used for reasoning, summarization, and checklist generation through Vertex AI

### Multi-Agent Workflow

- **Weather Agent**: Adds contextual seasonal hazards for winter
- **Manual Review Agent**: Extracts vehicle-specific winterization steps from the owner's manual
- **Recommendation Agent**: Compiles a structured winterization checklist in Markdown

This implementation serves as a reusable, extensible template for multi-agent RAG systems grounded in domain documents — integrating LLM reasoning with structured storage and enterprise-ready deployment.


```mermaid
flowchart TD
    A[User uploads PDF manual] --> B[Chunk and embed PDF content as 768 dimensional vectors]
    B --> C[Upload embeddings to Vertex AI Vector Index]
    C --> D[Create public index endpoint in us-central1]

    subgraph Vertex AI
        C
        D
    end

    E[User asks winter-related question] --> F[Query Vertex AI Matching Engine]
    F --> G[Retrieve relevant chunks from index]

    G --> H[Gemini LLM - Crew AI generates response]
    H --> I[Show final answer to user]

    %% Crew AI Integration
    subgraph Crew AI Agents
        J[Weather Agent]
        K[Manual Review Agent]
        L[Recommendation Agent]
        M[Page Citation Agent]
    end

    J --> N[Add seasonal context]
    G --> K
    K --> L
    L --> M
    M --> H

In [ ]:
# 🧱 Cell 1: Install packages, load environment, and initialize Vertex AI

# Uncomment if packages haven't been installed yet
# !pip install --upgrade google-cloud-aiplatform python-dotenv vertexai

import os
from dotenv import load_dotenv, find_dotenv
from google.cloud import aiplatform
import vertexai

# Load .env and set ADC (Application Default Credentials)
load_dotenv(find_dotenv(), override=True)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GCP_KEY_PATH")

# Initialize Vertex AI
project_id = os.getenv("GCP_PROJECT_ID")
region = os.getenv("VERTEX_REGION")
vertexai.init(project=project_id, location=region)
aiplatform.init(project=project_id, location=region)

print(f"✅ Vertex AI initialized with project: {project_id}, region: {region}")

In [ ]:
# 🧱 Cell 2: Load and chunk the Acura MDX 2022 user manual PDF

import fitz  # PyMuPDF
import nltk
from pathlib import Path
from nltk.tokenize import PunktSentenceTokenizer

# Load environment variable for the PDF
pdf_local_path = os.getenv("PDF_LOCAL_PATH")
pdf_path = Path(pdf_local_path)

# Download tokenizer model
print("📦 Downloading NLTK punkt model...")
nltk.download("punkt")

# Explicitly load tokenizer
tokenizer = PunktSentenceTokenizer()

# Open the PDF
doc = fitz.open(pdf_path)
print(f"📄 Loaded PDF: {pdf_path.name} with {len(doc)} pages")

# Extract text and tokenize into sentences
all_sentences = []
for i, page in enumerate(doc):
    text = page.get_text()
    sentences = tokenizer.tokenize(text)
    all_sentences.extend(sentences)
print(f"🧠 Extracted {len(all_sentences)} sentences from all pages")

# Group into ~500-word chunks
chunks = []
chunk = ""
for sentence in all_sentences:
    if len(chunk.split()) + len(sentence.split()) <= 500:
        chunk += " " + sentence
    else:
        chunks.append(chunk.strip())
        chunk = sentence
if chunk:
    chunks.append(chunk.strip())

print(f"✅ Final chunk count: {len(chunks)} (each ≤ ~500 words)")

In [ ]:
# 🧱 Cell 3: Generate embeddings for PDF chunks using text-embedding-005

from vertexai.preview.language_models import TextEmbeddingModel
import tqdm

# Initialize the embedding model (768-dim output)
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")

# Embed chunks in batches to avoid memory issues
embedded_chunks = []
batch_size = 20  # tweak if needed
for i in tqdm.trange(0, len(chunks), batch_size, desc="🔢 Embedding chunks"):
    batch = chunks[i:i+batch_size]
    embeddings = embedding_model.get_embeddings(batch)
    embedded_chunks.extend([e.values for e in embeddings])

print(f"✅ Embedded {len(embedded_chunks)} chunks.")

In [ ]:
# 🧱 Cell 4 (Updated): Create new Vertex AI Tree-AH Index with 768-dim and neighbors count

from google.cloud import aiplatform

# Init Vertex AI SDK
aiplatform.init(project=os.getenv("GCP_PROJECT_ID"), location=os.getenv("VERTEX_REGION"))

new_index_display_name = "acura-mdx-index-v3-768-dim"
dimension = 768

# ✅ Create index with required approximateNeighborsCount
vs_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=new_index_display_name,
    dimensions=dimension,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    shard_size="SHARD_SIZE_SMALL",
    index_update_method="STREAM_UPDATE",
    approximate_neighbors_count=150,  # Required for tree-AH
)

print(f"✅ Index created: {vs_index.display_name}")
print(f"🔗 Resource name: {vs_index.resource_name}")

In [ ]:
# 🧱 Cell 5: Create a new public Vertex AI Matching Engine endpoint

from google.cloud import aiplatform

# Define display name for the endpoint
endpoint_display_name = "acura-mdx-endpoint-v3-768-dim"

# Create the endpoint (public)
vs_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=endpoint_display_name,
    public_endpoint_enabled=True,
)

print(f"✅ Endpoint created: {vs_endpoint.display_name}")
print(f"🔗 Resource name: {vs_endpoint.resource_name}")

In [ ]:
# 🧱 Cell 6: Deploy index to endpoint with versioned index ID
#LONG Running process !!!

# Already available: vs_index and vs_endpoint from previous cells

deployed_index_id = "acura_mdx_v3_768_dim"

vs_deployed_index = vs_endpoint.deploy_index(
    index=vs_index,
    deployed_index_id=deployed_index_id,
    display_name=vs_index.display_name,
    machine_type="e2-standard-16",
    min_replica_count=1,
    max_replica_count=1,
)

print(f"✅ Deployed index '{vs_index.display_name}' to endpoint '{vs_endpoint.display_name}'")
print(f"🔗 Deployed Index ID: {deployed_index_id}")

In [ ]:
load_dotenv(find_dotenv(), override=True)
gcs_bucket_name_v2 = os.getenv("GCS_BUCKET_NAME_V2")
print(f"gcs_bucket_name_v2: {gcs_bucket_name_v2}")

In [ ]:
# 🧱 Cell 7A: Upload to Vertex Vector Index using LlamaIndex with explicit credentials

from google.oauth2 import service_account
from llama_index.embeddings.vertex import VertexTextEmbedding
from llama_index.vector_stores.vertexaivectorsearch import VertexAIVectorStore
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext

# Load from .env
load_dotenv(find_dotenv(), override=True)
project_id = os.getenv("GCP_PROJECT_ID")
gcs_bucket_name_v2 = os.getenv("GCS_BUCKET_NAME_V2")

# Load Vertex credentials securely
key_path = os.getenv("GCP_KEY_PATH")
credentials = service_account.Credentials.from_service_account_file(key_path)

# Init embedding model
embed_model = VertexTextEmbedding(
    model_name="text-embedding-005",
    project=project_id,
    location="us-central1",
    credentials=credentials,
)

# Setup vector store
vector_store = VertexAIVectorStore(
    project_id=project_id,
    region="us-central1",
    index_id=vs_index.resource_name,
    endpoint_id=vs_endpoint.resource_name,
    gcs_bucket_name=gcs_bucket_name_v2,
)

# Convert chunks to TextNodes
nodes = []
for i, (text, embedding) in enumerate(zip(chunks, embedded_chunks)):
    node = TextNode(
        id_=f"chunk-{i}",
        text=text,
        embedding=embedding,
        metadata={"source": "Acura MDX Manual", "chunk_index": i}
    )
    nodes.append(node)

# Upload to vector store
vector_store.add(nodes)

print(f"✅ Uploaded {len(nodes)} chunks to Vertex AI Vector Store using LlamaIndex")

In [ ]:
# !pip install "google-cloud-aiplatform[preview]<1.47"

In [ ]:
# 🧱 Cell 8: Setup LlamaIndex RAG query engine (Vertex Vector Store + Gemini Pro)

from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.vertex import Vertex

# Initialize Gemini LLM for completion
llm = Vertex(
    model="gemini-pro",
    project=project_id,
    location="us-central1",
    credentials=credentials,
    temperature=0.2,
    max_tokens=1024,
)

# Assign global settings
Settings.llm = llm
Settings.embed_model = embed_model  # From previous cell

# Set up storage context from vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index from storage
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model
)

# Setup retriever-backed query engine
query_engine = index.as_query_engine()

# Ask your question!
response = query_engine.query("How do I adjust the seat in the 2022 Acura MDX?")

# Show result
print("📘 Gemini Response:")
print("-" * 80)
print(response.response)

In [55]:
from crewai import LLM
from google.oauth2 import service_account
import os

# Load credentials
gcp_key_path = os.getenv("GCP_KEY_PATH")
gcp_project = os.getenv("GCP_PROJECT_ID")

# Secure credentials object
gcp_vertex_ai_credentials = service_account.Credentials.from_service_account_file(gcp_key_path)

# ✅ Initialize Gemini LLM via Vertex AI
gemini_llm = LLM(
    model='vertex_ai/gemini-2.0-flash',
    vertex_credentials=gcp_vertex_ai_credentials,
    vertex_project=gcp_project
)

print("✅ CrewAI Gemini LLM Initialized (via Vertex)")

✅ CrewAI Gemini LLM Initialized (via Vertex)


In [56]:
from crewai import Agent

# Agent 1: Weather Context Provider
weather_agent = Agent(
    role="Weather Agent",
    goal="Add relevant seasonal context for winter preparation.",
    backstory="A weather-focused AI agent specializing in identifying the effects of cold weather on vehicles.",
    llm=gemini_llm,
    verbose=True
)

# Agent 2: Acura Manual Expert
manual_review_agent = Agent(
    role="Manual Review Agent",
    goal="Review the Acura MDX 2022 owner's manual and find relevant information about seasonal or winter use.",
    backstory="An expert in parsing owner manuals to surface relevant operational and maintenance details.",
    llm=gemini_llm,
    verbose=True
)

# Agent 3: Vehicle Winterization Advisor
recommendation_agent = Agent(
    role="Recommendation Agent",
    goal="Suggest actions for winterizing a 2022 Acura MDX using Gemini and context from the manual.",
    backstory="A certified automotive assistant trained in seasonal readiness checks, with expertise in snow, battery care, tire selection, and cold weather maintenance.",
    llm=gemini_llm,
    verbose=True
)

print("✅ Crew AI Winterization Agents Successfully Defined")

✅ Crew AI Winterization Agents Successfully Defined


In [57]:
from crewai import Task

# Task 1: Add seasonal context (Winter)
add_seasonal_context_task = Task(
    description=(
        "Describe key winter weather conditions (cold, snow, ice, low visibility) "
        "that might affect a vehicle's operation, and provide a brief summary of why seasonal adjustments are necessary."
    ),
    agent=weather_agent,
    expected_output="A short winter-specific context paragraph about how weather impacts vehicle operation."
)

# Task 2: Review Acura MDX Manual for Winter Advice
manual_insights_task = Task(
    description=(
        "Based on the seasonal context, identify any winter-related maintenance or operational details from the 2022 Acura MDX owner's manual. "
        "This includes battery care, tire pressure, washer fluids, climate controls, and safety features that assist in cold weather."
    ),
    agent=manual_review_agent,
    expected_output="A bullet list of Acura MDX manual insights relevant to winterizing the vehicle."
)

# Task 3: Suggest Winterization Recommendations
generate_recommendations_task = Task(
    description=(
        "Using the seasonal context and manual insights, suggest a list of recommended steps to winterize a 2022 Acura MDX. "
        "Cover mechanical checks, safety checks, comfort features, and environmental preparations. Format in Markdown checklist."
    ),
    agent=recommendation_agent,
    expected_output="A Markdown-formatted winterization checklist for the Acura MDX."
)

print("✅ Winterization Tasks Successfully Defined")

✅ Winterization Tasks Successfully Defined


In [59]:
from crewai import Crew

# Create Crew AI pipeline
winterization_crew = Crew(
    agents=[weather_agent, manual_review_agent, recommendation_agent],
    tasks=[add_seasonal_context_task, manual_insights_task, generate_recommendations_task],
    verbose=True
)

# Run the workflow
print("🚀 Running Crew AI: Acura MDX Winterization Workflow...\n")
crew_output = winterization_crew.kickoff()

# Post-process Crew output to fix markdown rendering
clean_output = crew_output.raw.strip()
if clean_output.startswith("```markdown"):
    clean_output = clean_output.replace("```markdown", "").strip()
if clean_output.endswith("```"):
    clean_output = clean_output[:-3].strip()

from IPython.display import Markdown, display
display(Markdown(clean_output))

Overriding of current TracerProvider is not allowed


🚀 Running Crew AI: Acura MDX Winterization Workflow...

# Agent: Weather Agent
## Task: Describe key winter weather conditions (cold, snow, ice, low visibility) that might affect a vehicle's operation, and provide a brief summary of why seasonal adjustments are necessary.


# Agent: Weather Agent
## Final Answer: 
Winter weather presents a multitude of challenges for vehicles, demanding careful preparation and adjustments to ensure safe and reliable operation. The impact of cold temperatures, snow, ice, and reduced visibility can significantly affect a vehicle's performance and handling.

**Key Winter Weather Conditions and Their Effects on Vehicles:**

*   **Cold Temperatures:**
    *   **Battery Performance:** Cold reduces battery capacity, potentially leading to starting problems or complete failure.
    *   **Fluid Viscosity:** Oil and other fluids thicken, increasing engine wear and reducing fuel efficiency.
    *   **Tire Pressure:** Tire pressure decreases in cold weather, affec

## 2022 Acura MDX Winterization Checklist

**Mechanical Checks:**

*   [ ] **Battery Inspection:**
    *   [ ] Check battery terminals for corrosion and clean if necessary.
    *   [ ] Test battery's cold-cranking amps (CCA) and overall condition. Consider replacing if weak.
*   [ ] **Engine Oil:**
    *   [ ] Verify the engine oil viscosity is appropriate for cold temperatures (refer to the owner's manual for recommendations). Consider changing the oil and filter if needed.
*   [ ] **Coolant System:**
    *   [ ] Check coolant level in the reserve tank.
    *   [ ] Verify coolant concentration using a coolant tester (50/50 mixture of Acura Long Life Coolant Type 2 and distilled water is recommended).
    *   [ ] Inspect hoses for cracks, leaks, or damage.
*   [ ] **Braking System:**
    *   [ ] Inspect brake pads and rotors for wear.
    *   [ ] Check brake fluid level and condition.
*   [ ] **Tires:**
    *   [ ] Check tire pressure regularly, especially with temperature changes. Inflate to the pressure specified on the tire placard.
    *   [ ] Inspect tire tread depth. Consider winter tires for optimal snow and ice traction.
*   [ ] **Windshield Wiper Blades:**
    *   [ ] Inspect wiper blades for wear and replace if necessary.

**Safety Checks:**

*   [ ] **Lights:**
    *   [ ] Check all lights (headlights, taillights, brake lights, turn signals, and fog lights) to ensure they are working properly.
*   [ ] **Windshield Washer Fluid:**
    *   [ ] Fill the windshield washer fluid reservoir with a winter-specific fluid that contains antifreeze.
*   [ ] **Defroster:**
    *   [ ] Test the defroster to ensure it clears ice and fog from the windshield and windows effectively. Make sure the A/C is on.
*   [ ] **Vehicle Stability Assist (VSA):**
    *   [ ] Understand how VSA helps maintain control in slippery conditions (refer to the owner's manual).
*   [ ] **All-Wheel Drive (AWD) (If Equipped):**
    *   [ ] Understand how the AWD system enhances traction in snow and ice (refer to the owner's manual).
*   [ ] **Anti-lock Braking System (ABS):**
        *   [ ] Understand how the ABS system helps to prevent wheel lockup during braking, especially on slippery surfaces (refer to the owner's manual). Remember to apply firm, steady pressure in an emergency.

**Comfort Features:**

*   [ ] **Heated Seats and Steering Wheel (If Equipped):**
    *   [ ] Verify proper operation of heated seats and steering wheel.
*   [ ] **Remote Engine Starter (If Equipped):**
    *   [ ] Familiarize yourself with the operation and safety precautions of the remote engine starter.

**Environmental Preparations:**

*   [ ] **Emergency Kit:**
    *   [ ] Prepare an emergency kit containing:
        *   [ ] Jumper cables
        *   [ ] First-aid kit
        *   [ ] Flashlight
        *   [ ] Warm blanket
        *   [ ] Gloves
        *   [ ] Ice scraper
        *   [ ] Small shovel
        *   [ ] Sand or kitty litter (for traction)
        *   [ ] Warning flares or reflective triangles
*   [ ] **Ice Scraper and Snow Brush:**
    *   [ ] Keep an ice scraper and snow brush in the vehicle.

**Other Considerations:**

*   [ ] **Review Owner's Manual:**
    *   [ ] Consult the owner's manual for specific recommendations and procedures related to winter driving and maintenance for your 2022 Acura MDX.
*   [ ] **Tire Chains:**
    *   [ ] Determine if tire chains are needed for your typical winter driving conditions. If so, purchase Acura-approved chains and practice installing them before you need them. Install them on the front tires only, following the manufacturer's instructions.
*   [ ] **Driving Habits:**
    *   [ ] Adjust driving habits for winter conditions: reduce speed, increase following distance, brake gently, and be aware of black ice.
*   [ ] **Clear Vehicle:**
    *   [ ] Always clear all snow and ice from the vehicle (including roof, lights, and windows) before driving.